In [1]:
import os
from keras.models import *
from keras.optimizers import *
from keras.layers import *
import numpy as np
import sys
from DataProcessor import *

#the main and only function of the RNN_Model

def ApplyRNNModel(X,Y):

    #hidden layers and batch sizes
    HIDDEN_SIZE1 = 512
    HIDDEN_SIZE2 = 256
    BATCH_SIZE = 32


    print('Build model...')

    #use LSTM layers
    RNN = LSTM
    model = Sequential()

    #First RNN layer with input (number of collectors * len of the sequence)
    model.add(RNN(HIDDEN_SIZE1, input_shape=(len(X[0]), len(X[0][0])),activation='relu'))
    model.add(Reshape(target_shape=(16,32)))
    #Second RNN layer with the same input again
    model.add(RNN(HIDDEN_SIZE2, input_shape=(len(X[0]), len(X[0][0])),activation='relu'))
    #model.add(Dropout(0.4))
    #last Fully connected layer
    model.add(Dense(HIDDEN_SIZE2,activation='relu'))
    model.add(Dense(len(Y[0])))
    #Softmax to select based on probabilities
    model.add(Activation('softmax'))

    #optimize using gradient descent
    sgd = SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #to view a summary that represents the full RNN model
    model.summary()


    #next line: to be called if a model is saved with the same attributes
    model.load_weights('model')

    #run model on data (1000 times) with random 1% to validate the model
    model.fit(X, Y, batch_size=BATCH_SIZE, nb_epoch=100, validation_split=0.1)

    #built in keras function to save weights
    model.save_weights('model')

    classes = model.predict_classes(X)
    # print Y_train[3]

    with open('output.txt', 'w') as writer:
        for i in range(0, len(X)):
            true = Y[i]
            predicted = classes[i]
            writer.write("%s \t %s" % (predicted, np.argmax(true)))
            writer.write("\n")

#optional to normaize data (unifrom distributed)
def AugmentData(X,Y):

    X_New=[]
    Y_New=[]

    #sort depending on the target
    for i in range(len(X)):
        for j in range(len(X)-i-1):
            if np.argmax(Y[j]) < np.argmax(Y[j+1]):
                a,b=j,j+1
                X[a],X[b]=X[b],X[a]
                Y[a],Y[b]=Y[b],Y[a]

    #get min & max count of targets
    min=1000
    max=0
    count=1
    #loop on count of all targets
    for i in range(1,len(X)):
        if np.argmax(Y[i]) != np.argmax(Y[i-1]):
            #check if less than min or greater than max
            if count<min:
                min=count
            elif count>max:
                max=count
            count=1
        else:
            count+=1




    print 'Min Target:',min
    print 'Max Target:',max


    #augment Data
    count=1
    Len=len(X)
    for i in range(1,Len):
        if np.argmax(Y[i]) != np.argmax(Y[i-1]):
            #check if less than min or greater than max
            if count<max:
                for j in range(count,max):
                    X.append(X[i-1])
                    Y.append(Y[i-1])

            count=1
        else:
            count+=1

    if count < max:
        for j in range(count, max):
            X.append(X[i - 1])
            Y.append(Y[i - 1])

    return X,Y


#the RNN_Model start running here
processor=DataProcessor() #create object
processor.Generate_X_Y_Vectors(path='Data.csv',learn=1) #call generation function from the data processor file
X,Y=processor.X,processor.Y #get X,Y matrices from the processor object
#optional to augment data (unifrom distributed)
X,Y=AugmentData(X,Y)

ApplyRNNModel(X,Y) #call the RNN function














